In [4]:
!pip install -q -U google-genai


In [1]:
import socket
import requests.packages.urllib3.util.connection as urllib3_cn

# Garder une référence à la fonction originale
_original_getaddrinfo = socket.getaddrinfo

def patched_getaddrinfo(*args, **kwargs):
    """Force la résolution d'adresses en IPv4."""
    responses = _original_getaddrinfo(*args, **kwargs)
    # Filtrer pour ne garder que les adresses IPv4 (socket.AF_INET)
    return [res for res in responses if res[0] == socket.AF_INET]

# Appliquer le patch globalement pour la résolution DNS via socket
socket.getaddrinfo = patched_getaddrinfo

# Appliquer aussi le patch pour urllib3 (utilisé par certaines bibliothèques http)
# Cela peut aider si d'autres appels HTTP indirects sont faits, bien que
# le problème principal ici soit gRPC.
urllib3_cn.allowed_gai_family = lambda: socket.AF_INET

print("Patch appliqué pour forcer l'utilisation d'IPv4.")

Patch appliqué pour forcer l'utilisation d'IPv4.


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
import os
from dotenv import load_dotenv

# Charger les variables depuis le fichier .env dans l'environnement du processus Python actuel
load_dotenv()

# Maintenant, essayez de lire la variable d'environnement
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("La variable d'environnement GOOGLE_API_KEY n'a pas pu être chargée (vérifiez le fichier .env).")
else:
    print("Clé API chargée avec succès via le fichier .env.")
    print(f"Clé API : {GOOGLE_API_KEY[:4]}...")
    # Configurez genai ici, maintenant que la clé est chargée
    try:
        import google.generativeai as genai
        genai.configure(api_key=GOOGLE_API_KEY)
        print("Configuration de GenAI réussie.")
    except ImportError:
        print("Erreur: Le module google.generativeai n'est pas installé ou importé.")
    except Exception as e:
        print(f"Erreur lors de la configuration de genai : {e}")

Clé API chargée avec succès via le fichier .env.
Clé API : AIza...
Configuration de GenAI réussie.


In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-e

In [6]:
model = genai.GenerativeModel('models/gemini-2.5-pro-exp-03-25')

In [7]:

response = model.generate_content("What is the meaning of life?")

In [8]:
to_markdown(response.text)

> That's arguably *the* fundamental question humans have pondered across cultures and millennia! There's **no single, universally agreed-upon answer**, and the meaning of life is often considered subjective and deeply personal.
> 
> However, we can explore the different ways people approach this question:
> 
> **1. Philosophical Perspectives:**
> 
> *   **Nihilism:** Argues that life has no inherent, objective meaning, purpose, or intrinsic value.
> *   **Existentialism:** Agrees there's no pre-ordained meaning, but emphasizes that individuals are free and responsible for *creating* their own meaning through their choices and actions. (Think Sartre, Camus).
> *   **Hedonism:** Suggests the meaning of life is to maximize pleasure and minimize pain.
> *   **Stoicism:** Focuses on living virtuously, in accordance with reason and nature, accepting what we cannot control. Meaning comes from inner resilience and ethical action.
> *   **Humanism:** Emphasizes human potential, reason, ethics, and flourishing in this life, often focusing on contributing to the greater good of humanity without recourse to the supernatural.
> 
> **2. Religious & Spiritual Perspectives:**
> 
> *   **Theistic Religions (e.g., Christianity, Islam, Judaism):** Often propose that meaning comes from serving a higher power (God), fulfilling divine commandments, achieving salvation or enlightenment, and living in relationship with the creator and fellow beings.
> *   **Eastern Religions (e.g., Buddhism, Hinduism):** May focus on achieving enlightenment, breaking the cycle of rebirth (samsara), understanding one's true nature (Atman/Buddha-nature), living in harmony, and reducing suffering through compassion and detachment.
> *   **Spirituality (Broader sense):** Finding meaning through connection to something larger than oneself – nature, the universe, a collective consciousness, or an inner sense of purpose.
> 
> **3. Scientific & Biological Perspectives:**
> 
> *   From a purely biological standpoint, the "purpose" of life is survival and reproduction – passing on genetic material.
> *   Some scientists and thinkers find meaning in the universe's capacity to develop complexity and consciousness – seeing humans as a way the universe experiences itself.
> *   However, science primarily describes *how* life works, not necessarily its subjective *meaning* or ultimate *why*.
> 
> **4. Psychological & Personal Perspectives:**
> 
> *   **Finding Purpose:** Many find meaning through having clear goals, pursuing passions, or dedicating themselves to a cause larger than themselves.
> *   **Connection & Relationships:** Love, family, friendship, and community provide profound meaning for many people.
> *   **Contribution & Service:** Making a positive impact on others or the world.
> *   **Growth & Learning:** Continuously developing oneself, gaining knowledge and wisdom.
> *   **Experience & Appreciation:** Finding meaning in the simple act of experiencing life, beauty, joy, and even overcoming challenges.
> *   **Legacy:** Creating something lasting or influencing future generations.
> *   **Logotherapy (Viktor Frankl):** Argues that the primary human drive is not pleasure, but the discovery and pursuit of what we personally find meaningful, which can be found through work, love, and courage in suffering.
> 
> **In Conclusion:**
> 
> Instead of a single answer, the "meaning of life" might be:
> 
> *   **Something you discover:** Through introspection, experience, or faith.
> *   **Something you create:** Through your choices, actions, and commitments.
> *   **A combination of both.**
> *   **Found in the journey itself:** The process of living, learning, connecting, and seeking might be where the meaning lies, rather than a final destination or definition.
> 
> Ultimately, the question "What is the meaning of *my* life?" is one that each individual must grapple with and answer for themselves. What gives *your* life meaning and purpose?